# Deep Learning model

In [1]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [3]:
# Set features. This will also be used as your x values.
# I only select Transit Properties as listed here: 
# https://exoplanetarchive.ipac.caltech.edu/docs/API_kepcandidate_columns.html#transit_prop
selected_features = df[['koi_period', 'koi_time0bk', 'koi_impact', 'koi_duration', 
                        'koi_depth', 'koi_prad', 'koi_teq', 'koi_insol']]
X = selected_features.values
print(X.shape)

(6991, 8)


# Create a Train Test Split

Use `koi_disposition` for the y values

In [4]:
# prepare y values
y = df['koi_disposition'].values

# Step 1: Label-encode data set
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(y)
encoded_y = label_encoder.transform(y)

# Step 2: One-hot encoding
from keras.utils import to_categorical
one_hot_y = to_categorical(encoded_y)
one_hot_y


array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, one_hot_y, random_state=1)

In [6]:
X_train

array([[1.05484133e+01, 1.39064020e+02, 1.01700000e+00, ...,
        3.89000000e+00, 8.99000000e+02, 1.54840000e+02],
       [2.47543849e+01, 1.40207320e+02, 7.09000000e-01, ...,
        2.10000000e+00, 4.91000000e+02, 1.37000000e+01],
       [1.05733568e+00, 1.31792007e+02, 2.62000000e-01, ...,
        1.45900000e+01, 1.27600000e+03, 6.23510000e+02],
       ...,
       [1.07027283e+00, 1.31835891e+02, 1.26900000e+00, ...,
        2.87300000e+01, 1.22400000e+03, 5.28520000e+02],
       [1.87464901e+01, 1.47130740e+02, 5.08000000e-01, ...,
        3.05000000e+00, 6.86000000e+02, 5.23800000e+01],
       [2.10907909e+01, 1.33963160e+02, 8.32000000e-01, ...,
        1.88000000e+00, 6.06000000e+02, 3.18600000e+01]])

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [7]:
# Scale your data
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler().fit(X_train)

In [8]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# no need to scale y since it is already categorical
y_train

array([[1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       ...,
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

# Train the Model



In [13]:
# define the model
from tensorflow.keras.models import Sequential
model = Sequential()

from tensorflow.keras.layers import Dense
number_inputs = 8
number_hidden_nodes = 6
model.add(Dense(units=number_hidden_nodes, activation='relu', input_dim=number_inputs))
number_classes = 3
model.add(Dense(units=number_classes, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 6)                 54        
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 21        
Total params: 75
Trainable params: 75
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compile the Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
# Train the model
model.fit(X_train_scaled, y_train, epochs=1000, shuffle=True, verbose=2)

Epoch 1/1000
164/164 - 0s - loss: 0.9614 - accuracy: 0.4972
Epoch 2/1000
164/164 - 0s - loss: 0.8994 - accuracy: 0.5331
Epoch 3/1000
164/164 - 0s - loss: 0.8673 - accuracy: 0.5695
Epoch 4/1000
164/164 - 0s - loss: 0.8483 - accuracy: 0.6054
Epoch 5/1000
164/164 - 0s - loss: 0.8355 - accuracy: 0.6183
Epoch 6/1000
164/164 - 0s - loss: 0.8267 - accuracy: 0.6225
Epoch 7/1000
164/164 - 0s - loss: 0.8194 - accuracy: 0.6245
Epoch 8/1000
164/164 - 0s - loss: 0.8143 - accuracy: 0.6265
Epoch 9/1000
164/164 - 0s - loss: 0.8101 - accuracy: 0.6285
Epoch 10/1000
164/164 - 0s - loss: 0.8069 - accuracy: 0.6313
Epoch 11/1000
164/164 - 0s - loss: 0.8040 - accuracy: 0.6300
Epoch 12/1000
164/164 - 0s - loss: 0.8015 - accuracy: 0.6321
Epoch 13/1000
164/164 - 0s - loss: 0.7991 - accuracy: 0.6340
Epoch 14/1000
164/164 - 0s - loss: 0.7977 - accuracy: 0.6370
Epoch 15/1000
164/164 - 0s - loss: 0.7953 - accuracy: 0.6363
Epoch 16/1000
164/164 - 0s - loss: 0.7936 - accuracy: 0.6355
Epoch 17/1000
164/164 - 0s - loss

Epoch 135/1000
164/164 - 0s - loss: 0.7547 - accuracy: 0.6512
Epoch 136/1000
164/164 - 0s - loss: 0.7541 - accuracy: 0.6506
Epoch 137/1000
164/164 - 0s - loss: 0.7537 - accuracy: 0.6504
Epoch 138/1000
164/164 - 0s - loss: 0.7534 - accuracy: 0.6527
Epoch 139/1000
164/164 - 0s - loss: 0.7531 - accuracy: 0.6523
Epoch 140/1000
164/164 - 0s - loss: 0.7528 - accuracy: 0.6513
Epoch 141/1000
164/164 - 0s - loss: 0.7524 - accuracy: 0.6521
Epoch 142/1000
164/164 - 0s - loss: 0.7526 - accuracy: 0.6542
Epoch 143/1000
164/164 - 0s - loss: 0.7522 - accuracy: 0.6508
Epoch 144/1000
164/164 - 0s - loss: 0.7523 - accuracy: 0.6512
Epoch 145/1000
164/164 - 0s - loss: 0.7512 - accuracy: 0.6513
Epoch 146/1000
164/164 - 0s - loss: 0.7515 - accuracy: 0.6529
Epoch 147/1000
164/164 - 0s - loss: 0.7512 - accuracy: 0.6500
Epoch 148/1000
164/164 - 0s - loss: 0.7512 - accuracy: 0.6508
Epoch 149/1000
164/164 - 0s - loss: 0.7511 - accuracy: 0.6527
Epoch 150/1000
164/164 - 0s - loss: 0.7508 - accuracy: 0.6553
Epoch 15

164/164 - 0s - loss: 0.7356 - accuracy: 0.6590
Epoch 268/1000
164/164 - 0s - loss: 0.7353 - accuracy: 0.6592
Epoch 269/1000
164/164 - 0s - loss: 0.7354 - accuracy: 0.6597
Epoch 270/1000
164/164 - 0s - loss: 0.7356 - accuracy: 0.6622
Epoch 271/1000
164/164 - 0s - loss: 0.7351 - accuracy: 0.6615
Epoch 272/1000
164/164 - 0s - loss: 0.7354 - accuracy: 0.6569
Epoch 273/1000
164/164 - 0s - loss: 0.7352 - accuracy: 0.6611
Epoch 274/1000
164/164 - 0s - loss: 0.7353 - accuracy: 0.6601
Epoch 275/1000
164/164 - 0s - loss: 0.7349 - accuracy: 0.6607
Epoch 276/1000
164/164 - 0s - loss: 0.7351 - accuracy: 0.6578
Epoch 277/1000
164/164 - 0s - loss: 0.7353 - accuracy: 0.6588
Epoch 278/1000
164/164 - 0s - loss: 0.7349 - accuracy: 0.6567
Epoch 279/1000
164/164 - 0s - loss: 0.7349 - accuracy: 0.6553
Epoch 280/1000
164/164 - 0s - loss: 0.7350 - accuracy: 0.6580
Epoch 281/1000
164/164 - 0s - loss: 0.7351 - accuracy: 0.6594
Epoch 282/1000
164/164 - 0s - loss: 0.7350 - accuracy: 0.6586
Epoch 283/1000
164/164 

Epoch 400/1000
164/164 - 0s - loss: 0.7312 - accuracy: 0.6599
Epoch 401/1000
164/164 - 0s - loss: 0.7305 - accuracy: 0.6620
Epoch 402/1000
164/164 - 0s - loss: 0.7310 - accuracy: 0.6594
Epoch 403/1000
164/164 - 0s - loss: 0.7309 - accuracy: 0.6609
Epoch 404/1000
164/164 - 0s - loss: 0.7311 - accuracy: 0.6595
Epoch 405/1000
164/164 - 0s - loss: 0.7307 - accuracy: 0.6601
Epoch 406/1000
164/164 - 0s - loss: 0.7310 - accuracy: 0.6584
Epoch 407/1000
164/164 - 0s - loss: 0.7308 - accuracy: 0.6607
Epoch 408/1000
164/164 - 0s - loss: 0.7310 - accuracy: 0.6599
Epoch 409/1000
164/164 - 0s - loss: 0.7309 - accuracy: 0.6613
Epoch 410/1000
164/164 - 0s - loss: 0.7308 - accuracy: 0.6592
Epoch 411/1000
164/164 - 0s - loss: 0.7309 - accuracy: 0.6599
Epoch 412/1000
164/164 - 0s - loss: 0.7309 - accuracy: 0.6613
Epoch 413/1000
164/164 - 0s - loss: 0.7307 - accuracy: 0.6584
Epoch 414/1000
164/164 - 0s - loss: 0.7306 - accuracy: 0.6599
Epoch 415/1000
164/164 - 0s - loss: 0.7306 - accuracy: 0.6603
Epoch 41

164/164 - 0s - loss: 0.7292 - accuracy: 0.6592
Epoch 533/1000
164/164 - 0s - loss: 0.7292 - accuracy: 0.6601
Epoch 534/1000
164/164 - 0s - loss: 0.7292 - accuracy: 0.6597
Epoch 535/1000
164/164 - 0s - loss: 0.7292 - accuracy: 0.6592
Epoch 536/1000
164/164 - 0s - loss: 0.7289 - accuracy: 0.6592
Epoch 537/1000
164/164 - 0s - loss: 0.7291 - accuracy: 0.6601
Epoch 538/1000
164/164 - 0s - loss: 0.7294 - accuracy: 0.6618
Epoch 539/1000
164/164 - 0s - loss: 0.7289 - accuracy: 0.6616
Epoch 540/1000
164/164 - 0s - loss: 0.7290 - accuracy: 0.6626
Epoch 541/1000
164/164 - 0s - loss: 0.7289 - accuracy: 0.6599
Epoch 542/1000
164/164 - 0s - loss: 0.7293 - accuracy: 0.6597
Epoch 543/1000
164/164 - 0s - loss: 0.7295 - accuracy: 0.6605
Epoch 544/1000
164/164 - 0s - loss: 0.7290 - accuracy: 0.6597
Epoch 545/1000
164/164 - 0s - loss: 0.7294 - accuracy: 0.6611
Epoch 546/1000
164/164 - 0s - loss: 0.7290 - accuracy: 0.6597
Epoch 547/1000
164/164 - 0s - loss: 0.7289 - accuracy: 0.6595
Epoch 548/1000
164/164 

Epoch 665/1000
164/164 - 0s - loss: 0.7266 - accuracy: 0.6626
Epoch 666/1000
164/164 - 0s - loss: 0.7267 - accuracy: 0.6636
Epoch 667/1000
164/164 - 0s - loss: 0.7267 - accuracy: 0.6609
Epoch 668/1000
164/164 - 0s - loss: 0.7270 - accuracy: 0.6637
Epoch 669/1000
164/164 - 0s - loss: 0.7262 - accuracy: 0.6643
Epoch 670/1000
164/164 - 0s - loss: 0.7264 - accuracy: 0.6649
Epoch 671/1000
164/164 - 0s - loss: 0.7265 - accuracy: 0.6639
Epoch 672/1000
164/164 - 0s - loss: 0.7262 - accuracy: 0.6620
Epoch 673/1000
164/164 - 0s - loss: 0.7262 - accuracy: 0.6651
Epoch 674/1000
164/164 - 0s - loss: 0.7265 - accuracy: 0.6649
Epoch 675/1000
164/164 - 0s - loss: 0.7263 - accuracy: 0.6641
Epoch 676/1000
164/164 - 0s - loss: 0.7265 - accuracy: 0.6611
Epoch 677/1000
164/164 - 0s - loss: 0.7263 - accuracy: 0.6630
Epoch 678/1000
164/164 - 0s - loss: 0.7262 - accuracy: 0.6664
Epoch 679/1000
164/164 - 0s - loss: 0.7263 - accuracy: 0.6653
Epoch 680/1000
164/164 - 0s - loss: 0.7264 - accuracy: 0.6634
Epoch 68

164/164 - 0s - loss: 0.7240 - accuracy: 0.6683
Epoch 798/1000
164/164 - 0s - loss: 0.7240 - accuracy: 0.6641
Epoch 799/1000
164/164 - 0s - loss: 0.7243 - accuracy: 0.6658
Epoch 800/1000
164/164 - 0s - loss: 0.7237 - accuracy: 0.6664
Epoch 801/1000
164/164 - 0s - loss: 0.7238 - accuracy: 0.6656
Epoch 802/1000
164/164 - 0s - loss: 0.7240 - accuracy: 0.6647
Epoch 803/1000
164/164 - 0s - loss: 0.7235 - accuracy: 0.6691
Epoch 804/1000
164/164 - 0s - loss: 0.7237 - accuracy: 0.6700
Epoch 805/1000
164/164 - 0s - loss: 0.7237 - accuracy: 0.6674
Epoch 806/1000
164/164 - 0s - loss: 0.7240 - accuracy: 0.6683
Epoch 807/1000
164/164 - 0s - loss: 0.7236 - accuracy: 0.6660
Epoch 808/1000
164/164 - 0s - loss: 0.7238 - accuracy: 0.6641
Epoch 809/1000
164/164 - 0s - loss: 0.7236 - accuracy: 0.6666
Epoch 810/1000
164/164 - 0s - loss: 0.7233 - accuracy: 0.6681
Epoch 811/1000
164/164 - 0s - loss: 0.7233 - accuracy: 0.6668
Epoch 812/1000
164/164 - 0s - loss: 0.7236 - accuracy: 0.6668
Epoch 813/1000
164/164 

Epoch 930/1000
164/164 - 0s - loss: 0.7185 - accuracy: 0.6708
Epoch 931/1000
164/164 - 0s - loss: 0.7186 - accuracy: 0.6708
Epoch 932/1000
164/164 - 0s - loss: 0.7186 - accuracy: 0.6710
Epoch 933/1000
164/164 - 0s - loss: 0.7181 - accuracy: 0.6700
Epoch 934/1000
164/164 - 0s - loss: 0.7184 - accuracy: 0.6706
Epoch 935/1000
164/164 - 0s - loss: 0.7183 - accuracy: 0.6716
Epoch 936/1000
164/164 - 0s - loss: 0.7183 - accuracy: 0.6697
Epoch 937/1000
164/164 - 0s - loss: 0.7185 - accuracy: 0.6712
Epoch 938/1000
164/164 - 0s - loss: 0.7183 - accuracy: 0.6716
Epoch 939/1000
164/164 - 0s - loss: 0.7182 - accuracy: 0.6718
Epoch 940/1000
164/164 - 0s - loss: 0.7187 - accuracy: 0.6708
Epoch 941/1000
164/164 - 0s - loss: 0.7180 - accuracy: 0.6695
Epoch 942/1000
164/164 - 0s - loss: 0.7185 - accuracy: 0.6698
Epoch 943/1000
164/164 - 0s - loss: 0.7178 - accuracy: 0.6698
Epoch 944/1000
164/164 - 0s - loss: 0.7183 - accuracy: 0.6698
Epoch 945/1000
164/164 - 0s - loss: 0.7179 - accuracy: 0.6721
Epoch 94

In [16]:
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
# print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
# print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

55/55 - 0s - loss: 0.7184 - accuracy: 0.6728
Loss: 0.7184042930603027, Accuracy: 0.6727688908576965


## Deep Learning

In [17]:
# Deep learning
deep_model = Sequential()
deep_model.add(Dense(units=6, activation='relu', input_dim=number_inputs))
deep_model.add(Dense(units=6, activation='relu'))
deep_model.add(Dense(units=number_classes, activation='softmax'))
deep_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 6)                 54        
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 21        
Total params: 117
Trainable params: 117
Non-trainable params: 0
_________________________________________________________________


In [18]:
deep_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
deep_model.fit(X_train_scaled, y_train, epochs=1000, shuffle=True, verbose=2)

Epoch 1/1000
164/164 - 0s - loss: 1.0038 - accuracy: 0.4875
Epoch 2/1000
164/164 - 0s - loss: 0.9187 - accuracy: 0.5024
Epoch 3/1000
164/164 - 0s - loss: 0.8672 - accuracy: 0.5796
Epoch 4/1000
164/164 - 0s - loss: 0.8363 - accuracy: 0.6122
Epoch 5/1000
164/164 - 0s - loss: 0.8189 - accuracy: 0.6195
Epoch 6/1000
164/164 - 0s - loss: 0.8089 - accuracy: 0.6233
Epoch 7/1000
164/164 - 0s - loss: 0.8021 - accuracy: 0.6267
Epoch 8/1000
164/164 - 0s - loss: 0.7972 - accuracy: 0.6254
Epoch 9/1000
164/164 - 0s - loss: 0.7931 - accuracy: 0.6288
Epoch 10/1000
164/164 - 0s - loss: 0.7901 - accuracy: 0.6313
Epoch 11/1000
164/164 - 0s - loss: 0.7867 - accuracy: 0.6349
Epoch 12/1000
164/164 - 0s - loss: 0.7847 - accuracy: 0.6420
Epoch 13/1000
164/164 - 0s - loss: 0.7818 - accuracy: 0.6430
Epoch 14/1000
164/164 - 0s - loss: 0.7809 - accuracy: 0.6451
Epoch 15/1000
164/164 - 0s - loss: 0.7783 - accuracy: 0.6471
Epoch 16/1000
164/164 - 0s - loss: 0.7778 - accuracy: 0.6441
Epoch 17/1000
164/164 - 0s - loss

Epoch 135/1000
164/164 - 0s - loss: 0.7359 - accuracy: 0.6599
Epoch 136/1000
164/164 - 0s - loss: 0.7354 - accuracy: 0.6584
Epoch 137/1000
164/164 - 0s - loss: 0.7351 - accuracy: 0.6595
Epoch 138/1000
164/164 - 0s - loss: 0.7353 - accuracy: 0.6605
Epoch 139/1000
164/164 - 0s - loss: 0.7357 - accuracy: 0.6582
Epoch 140/1000
164/164 - 0s - loss: 0.7350 - accuracy: 0.6603
Epoch 141/1000
164/164 - 0s - loss: 0.7353 - accuracy: 0.6615
Epoch 142/1000
164/164 - 0s - loss: 0.7346 - accuracy: 0.6620
Epoch 143/1000
164/164 - 0s - loss: 0.7351 - accuracy: 0.6609
Epoch 144/1000
164/164 - 0s - loss: 0.7348 - accuracy: 0.6615
Epoch 145/1000
164/164 - 0s - loss: 0.7346 - accuracy: 0.6628
Epoch 146/1000
164/164 - 0s - loss: 0.7343 - accuracy: 0.6628
Epoch 147/1000
164/164 - 0s - loss: 0.7337 - accuracy: 0.6597
Epoch 148/1000
164/164 - 0s - loss: 0.7336 - accuracy: 0.6613
Epoch 149/1000
164/164 - 0s - loss: 0.7347 - accuracy: 0.6595
Epoch 150/1000
164/164 - 0s - loss: 0.7340 - accuracy: 0.6613
Epoch 15

164/164 - 0s - loss: 0.7232 - accuracy: 0.6624
Epoch 268/1000
164/164 - 0s - loss: 0.7228 - accuracy: 0.6674
Epoch 269/1000
164/164 - 0s - loss: 0.7216 - accuracy: 0.6632
Epoch 270/1000
164/164 - 0s - loss: 0.7224 - accuracy: 0.6676
Epoch 271/1000
164/164 - 0s - loss: 0.7217 - accuracy: 0.6672
Epoch 272/1000
164/164 - 0s - loss: 0.7218 - accuracy: 0.6653
Epoch 273/1000
164/164 - 0s - loss: 0.7210 - accuracy: 0.6643
Epoch 274/1000
164/164 - 0s - loss: 0.7207 - accuracy: 0.6656
Epoch 275/1000
164/164 - 0s - loss: 0.7200 - accuracy: 0.6630
Epoch 276/1000
164/164 - 0s - loss: 0.7201 - accuracy: 0.6649
Epoch 277/1000
164/164 - 0s - loss: 0.7194 - accuracy: 0.6643
Epoch 278/1000
164/164 - 0s - loss: 0.7183 - accuracy: 0.6679
Epoch 279/1000
164/164 - 0s - loss: 0.7185 - accuracy: 0.6645
Epoch 280/1000
164/164 - 0s - loss: 0.7182 - accuracy: 0.6649
Epoch 281/1000
164/164 - 0s - loss: 0.7180 - accuracy: 0.6674
Epoch 282/1000
164/164 - 0s - loss: 0.7185 - accuracy: 0.6653
Epoch 283/1000
164/164 

Epoch 400/1000
164/164 - 0s - loss: 0.7035 - accuracy: 0.6668
Epoch 401/1000
164/164 - 0s - loss: 0.7039 - accuracy: 0.6670
Epoch 402/1000
164/164 - 0s - loss: 0.7025 - accuracy: 0.6677
Epoch 403/1000
164/164 - 0s - loss: 0.7035 - accuracy: 0.6693
Epoch 404/1000
164/164 - 0s - loss: 0.7036 - accuracy: 0.6645
Epoch 405/1000
164/164 - 0s - loss: 0.7040 - accuracy: 0.6670
Epoch 406/1000
164/164 - 0s - loss: 0.7031 - accuracy: 0.6674
Epoch 407/1000
164/164 - 0s - loss: 0.7033 - accuracy: 0.6668
Epoch 408/1000
164/164 - 0s - loss: 0.7026 - accuracy: 0.6702
Epoch 409/1000
164/164 - 0s - loss: 0.7040 - accuracy: 0.6649
Epoch 410/1000
164/164 - 0s - loss: 0.7032 - accuracy: 0.6660
Epoch 411/1000
164/164 - 0s - loss: 0.7022 - accuracy: 0.6704
Epoch 412/1000
164/164 - 0s - loss: 0.7031 - accuracy: 0.6706
Epoch 413/1000
164/164 - 0s - loss: 0.7025 - accuracy: 0.6679
Epoch 414/1000
164/164 - 0s - loss: 0.7025 - accuracy: 0.6655
Epoch 415/1000
164/164 - 0s - loss: 0.7032 - accuracy: 0.6691
Epoch 41

164/164 - 0s - loss: 0.6960 - accuracy: 0.6695
Epoch 533/1000
164/164 - 0s - loss: 0.6967 - accuracy: 0.6685
Epoch 534/1000
164/164 - 0s - loss: 0.6971 - accuracy: 0.6710
Epoch 535/1000
164/164 - 0s - loss: 0.6963 - accuracy: 0.6700
Epoch 536/1000
164/164 - 0s - loss: 0.6976 - accuracy: 0.6689
Epoch 537/1000
164/164 - 0s - loss: 0.6965 - accuracy: 0.6687
Epoch 538/1000
164/164 - 0s - loss: 0.6964 - accuracy: 0.6716
Epoch 539/1000
164/164 - 0s - loss: 0.6966 - accuracy: 0.6721
Epoch 540/1000
164/164 - 0s - loss: 0.6959 - accuracy: 0.6723
Epoch 541/1000
164/164 - 0s - loss: 0.6962 - accuracy: 0.6702
Epoch 542/1000
164/164 - 0s - loss: 0.6967 - accuracy: 0.6662
Epoch 543/1000
164/164 - 0s - loss: 0.6970 - accuracy: 0.6697
Epoch 544/1000
164/164 - 0s - loss: 0.6962 - accuracy: 0.6697
Epoch 545/1000
164/164 - 0s - loss: 0.6961 - accuracy: 0.6735
Epoch 546/1000
164/164 - 0s - loss: 0.6956 - accuracy: 0.6718
Epoch 547/1000
164/164 - 0s - loss: 0.6960 - accuracy: 0.6691
Epoch 548/1000
164/164 

Epoch 665/1000
164/164 - 0s - loss: 0.6918 - accuracy: 0.6773
Epoch 666/1000
164/164 - 0s - loss: 0.6924 - accuracy: 0.6716
Epoch 667/1000
164/164 - 0s - loss: 0.6919 - accuracy: 0.6740
Epoch 668/1000
164/164 - 0s - loss: 0.6921 - accuracy: 0.6719
Epoch 669/1000
164/164 - 0s - loss: 0.6925 - accuracy: 0.6769
Epoch 670/1000
164/164 - 0s - loss: 0.6917 - accuracy: 0.6748
Epoch 671/1000
164/164 - 0s - loss: 0.6916 - accuracy: 0.6725
Epoch 672/1000
164/164 - 0s - loss: 0.6920 - accuracy: 0.6780
Epoch 673/1000
164/164 - 0s - loss: 0.6920 - accuracy: 0.6746
Epoch 674/1000
164/164 - 0s - loss: 0.6928 - accuracy: 0.6763
Epoch 675/1000
164/164 - 0s - loss: 0.6911 - accuracy: 0.6788
Epoch 676/1000
164/164 - 0s - loss: 0.6916 - accuracy: 0.6758
Epoch 677/1000
164/164 - 0s - loss: 0.6922 - accuracy: 0.6756
Epoch 678/1000
164/164 - 0s - loss: 0.6914 - accuracy: 0.6803
Epoch 679/1000
164/164 - 0s - loss: 0.6921 - accuracy: 0.6718
Epoch 680/1000
164/164 - 0s - loss: 0.6914 - accuracy: 0.6742
Epoch 68

164/164 - 0s - loss: 0.6886 - accuracy: 0.6811
Epoch 798/1000
164/164 - 0s - loss: 0.6888 - accuracy: 0.6822
Epoch 799/1000
164/164 - 0s - loss: 0.6892 - accuracy: 0.6792
Epoch 800/1000
164/164 - 0s - loss: 0.6889 - accuracy: 0.6788
Epoch 801/1000
164/164 - 0s - loss: 0.6887 - accuracy: 0.6811
Epoch 802/1000
164/164 - 0s - loss: 0.6893 - accuracy: 0.6780
Epoch 803/1000
164/164 - 0s - loss: 0.6887 - accuracy: 0.6782
Epoch 804/1000
164/164 - 0s - loss: 0.6891 - accuracy: 0.6763
Epoch 805/1000
164/164 - 0s - loss: 0.6887 - accuracy: 0.6815
Epoch 806/1000
164/164 - 0s - loss: 0.6890 - accuracy: 0.6771
Epoch 807/1000
164/164 - 0s - loss: 0.6877 - accuracy: 0.6828
Epoch 808/1000
164/164 - 0s - loss: 0.6892 - accuracy: 0.6777
Epoch 809/1000
164/164 - 0s - loss: 0.6890 - accuracy: 0.6815
Epoch 810/1000
164/164 - 0s - loss: 0.6892 - accuracy: 0.6801
Epoch 811/1000
164/164 - 0s - loss: 0.6893 - accuracy: 0.6782
Epoch 812/1000
164/164 - 0s - loss: 0.6896 - accuracy: 0.6759
Epoch 813/1000
164/164 

Epoch 930/1000
164/164 - 0s - loss: 0.6863 - accuracy: 0.6824
Epoch 931/1000
164/164 - 0s - loss: 0.6869 - accuracy: 0.6813
Epoch 932/1000
164/164 - 0s - loss: 0.6873 - accuracy: 0.6824
Epoch 933/1000
164/164 - 0s - loss: 0.6868 - accuracy: 0.6796
Epoch 934/1000
164/164 - 0s - loss: 0.6863 - accuracy: 0.6832
Epoch 935/1000
164/164 - 0s - loss: 0.6862 - accuracy: 0.6811
Epoch 936/1000
164/164 - 0s - loss: 0.6860 - accuracy: 0.6826
Epoch 937/1000
164/164 - 0s - loss: 0.6867 - accuracy: 0.6807
Epoch 938/1000
164/164 - 0s - loss: 0.6856 - accuracy: 0.6815
Epoch 939/1000
164/164 - 0s - loss: 0.6863 - accuracy: 0.6786
Epoch 940/1000
164/164 - 0s - loss: 0.6863 - accuracy: 0.6811
Epoch 941/1000
164/164 - 0s - loss: 0.6856 - accuracy: 0.6821
Epoch 942/1000
164/164 - 0s - loss: 0.6854 - accuracy: 0.6807
Epoch 943/1000
164/164 - 0s - loss: 0.6859 - accuracy: 0.6807
Epoch 944/1000
164/164 - 0s - loss: 0.6857 - accuracy: 0.6796
Epoch 945/1000
164/164 - 0s - loss: 0.6860 - accuracy: 0.6842
Epoch 94

In [20]:
deep_model_loss, deep_model_accuracy = deep_model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {deep_model_loss}, Accuracy: {deep_model_accuracy}")

55/55 - 0s - loss: 0.6966 - accuracy: 0.6825
Loss: 0.6966097950935364, Accuracy: 0.6824942827224731


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [21]:
# Create the GridSearchCV model

In [22]:
# Train the model with GridSearch

In [23]:
# print(grid2.best_params_)
# print(grid2.best_score_)

# Save the Model

In [24]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
# import joblib
# filename = 'jamesye.sav'
# joblib.dump(deep_model, filename)
model.save("deep_model.h5")